<font color = "#CC3D3D">
# End-to-End Machine Learning Project #
<p>
- #### *Based on CRISP-DM & scikit-learn*    
<br>
<img align="left" src="http://www.kdnuggets.com/wp-content/uploads/crisp-dm-4-problems-fig1.png" alt="CRISP-DM">

## Step 1: Business Understanding ##

1. Business Objectives
 - 새로운 개인연금상품(PEP: Personal Equity Plan)을 개발하여 기존 고객들을 대상으로 가능한 많은 계좌를 유치
2. Analytics Goals
 - PEP 가입 예측모형 개발
 - 고객 프로파일 개발
 - 다이렉트 메일 광고 효율성 제고
 - 타겟 메일링에 의한 응답률 제고 

## Step 2: Data Understanding ##
1. 데이터 획득 절차
 - 기존고객 DB로부터 시험메일 발송을 위한 표본고객목록을 추출
 - 새로운 금융상품(PEP)의 제안 메일을 발송
 - 고객의 반응을 기록
2. 분석 데이터
 - 학습용 데이터 600건
 - 신규고객 데이터 200건

In [2]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Collect Initial Data ###

##### for modeling

In [3]:
df = pd.read_csv("data_pepTestCustomers.csv")
df

,id,age,sex,region,income,married,children,car,save_act,current_act,mortgage,pep
0,ID12101,48.0,0,0,17546.00,0,1,0,0,0,0,1
1,ID12102,40.0,1,3,30085.10,1,3,1,0,1,1,0
2,ID12103,NaN,0,0,16575.40,1,0,1,1,1,0,0
3,ID12104,23.0,0,3,20375.40,1,3,0,0,1,0,0
4,ID12105,57.0,0,1,50576.30,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
595,ID12696,61.0,0,0,47025.00,0,2,1,1,1,1,0
596,ID12697,30.0,0,0,9672.25,1,0,1,1,1,0,0
597,ID12698,31.0,0,3,15976.30,1,0,1,1,0,0,1
598,ID12699,29.0,1,0,14711.80,1,0,0,1,0,1,0


##### for deployment

In [ ]:
new = pd.read_csv("data_pepNewCustomers.csv")
new.head()

### Describe Data ###

In [ ]:
# 데이터 구조 살펴보기
# ex) age의 경우 600개중 540개만 있어서 60개가 결측치로 예상
df.info()

In [ ]:
# 수치형 속성의 요약통계 보기

df.describe()

In [ ]:
# 모든 수치형 속성에 대한 히스토그램 한꺼번에 그리기
# 숫치형은 age와 income 나머지는 범주형이라 볼 수 있다
# bins = 30 은 x축을 30개로 쪼개라
df.hist(bins=30, figsize=(20,15))

In [ ]:
# 모형을 적용할 new 데이터에는 class 필드(즉, pep)가 없다.

display(df.shape) 
display(new.shape)

In [ ]:
df.shape
new.shape

### Explore Data ###

##### Look for Correlations #####
<img align="left" src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d4/Correlation_examples2.svg/506px-Correlation_examples2.svg.png" alt="correlation">

In [ ]:
# 속성 간 피어슨 상관계수 출력하기
# 전체 수치형 데이터를 콜러에이션으로 보여 줌
# 상관관계 예측변수를 줄여줌

corr = df.corr()
corr

In [ ]:
# 속성간 상관계수를 한눈에 파악하기 위해 그림으로 도식하기

plt.matshow(corr)

In [ ]:
# 클래스 변수인 pep와 각 속성간의 피어슨 상관계수를 내림차순으로 출력하기
# pep 데이터 sorting

corr.pep.sort_values(ascending=False)

##### Find Missing Values

In [ ]:
# 결측값이 존재하는 속성이 무엇인지, 몇개나 있는지 파악할 수 있는 첫번째 방법

df.info()

In [ ]:
# 결측값이 존재하는 속성이 무엇인지, 몇개나 있는지 파악할 수 있는 두번째 방법
# null 갯수 체크

df.isnull().sum()

##### Detect Outliers #####
<img align="left" src="http://www.whatissixsigma.net/wp-content/uploads/2015/07/Box-Plot-Diagram-to-identify-Outliers-figure-1.png" alt="Boxplot Outlier">

In [ ]:
# 나이와 수입 속성에 이상치가 있는지 Boxplot을 그려서 알아본다.
# 이상치 발굴은 boxplot에서 박스 밖에 있는 점을 이상치(Outlier)로 본다.

df.loc[:,['age','income']].plot.box(subplots=True, layout=(2,1), figsize=(5,5))

## Step 3: Data Preparation ##

### Clean Data ###
##### Replace Missing Values #####

In [ ]:
# Save original data 
# => 학습 데이터를 조작하다가 문제가 생겨 처음으로 되돌아 가려면 이 셀부터 다시 시작하세요 !!!
# 데이터 원본 유지를 이해 copy를 사용

mdf = df.copy()

In [ ]:
# 결측값이 있는 속성 파악

mdf.info()

In [ ]:
# 나이의 평균값과 나이별 발생빈도 계산하기
# 채워넣을 땐 평균 또는 최빈값(가장 많이 나오는 값)을 쓴다

display(mdf.age.mean())
display(mdf.age.value_counts().head())

In [ ]:
# 평균값으로 결측값 대체하기
# fillna 는 널값을 채우라. 여기서는 평균으로 채우고 있음.
# 분석의 상황에 따라 null 값을 그대로 사용함.
 
mdf.age.fillna(round(mdf.age.mean(),0), inplace=True)
mdf.info()

### Construct Data ###
##### Derive Attributes #####

In [ ]:
# 명목소득 보다 명목소득을 자녀수로 나눈 실질소득이 개인연금 가입에 더 유효한 속성이라고 가정하고 이를 구현
# 자식에 따라 realincome 변수를 추가

mdf['realincome'] = np.where(mdf['children']==0, mdf['income'], mdf['income']/mdf['children'])
mdf.head()

<font color = "blue">
***[numpy.where(condition, x, y)](https://docs.scipy.org/doc/numpy/reference/generated/numpy.where.html#numpy.where)***<br>
***True이면 x를 , 그렇지 않으면 y ***<br>                                 
*return elements, either from x or y, depending on condition*

### Select Data ###
##### Filter Attributes  #####

In [ ]:
# 파생변수인 실질소득을 유도하는데 사용된 명목소득과 자녀수를 제거
# pep와 상관관계가 거의 없는 current_act, car, mortgage, region 속성 제거
# 아래 컬럼은 없애고 분석하겠다.

columns = ['income', 'children', 'current_act', 'car', 'mortgage', 'region']
mdf = mdf.drop(columns, axis=1)
mdf.head()

### Split Data ###
<img align="left" src="https://www.developer.com/imagesvr_ce/6793/ML4.png" width=500 height=500 alt="Boxplot Outlier">

In [ ]:
# for Hold-out validation

from sklearn.model_selection import train_test_split  

In [ ]:
# scikit-learn에서는 속성과 클래스를 분리하여야 한다.

dfX = mdf.drop(['id','pep'], axis=1)  # exclude 'id' attribute & class variable
dfy = mdf['pep']                    # class variable
X_train, X_test, y_train, y_test = train_test_split(dfX, dfy, test_size=0.25, random_state=0)

In [ ]:
# 지정한 비율(75:25)로 데이터가 잘 나누어졌는지 확인

display(X_train.shape, X_test.shape)
X_train.head()

## Step 4: Modeling ##
<br>
<font color = "blue">
**Scikit-Learn Estimator Interface**
1. Import the estimator
2. Instantiate the estimator
3. Fit the data to the estimator: **.fit()**
4. Generate a prediction: **.predict()**
5. Evaluate the estimator: **.score()**
</font>
*ref) "Introduction to Machine Learning with Python", pp.254*

### Build Model ###

<font color = "darkgreen">
##### 1. Decision Trees #####
</font>
<img align="left" src="https://cdn-images-1.medium.com/max/870/1*xzF10JmR3K0rnZ8jtIHI_g.png" width=700 height=500 alt="Decision Tree">

<font color = "blue">
***[sklearn.tree.DecisionTreeClassifier()](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)***

In [ ]:
# 1. Import the estimator

from sklearn.tree import DecisionTreeClassifier

In [ ]:
# 2. Instantiate the estimator

tree = DecisionTreeClassifier(max_depth=6, random_state=0)

In [ ]:
# 3. Fit the data to the estimator

tree.fit(X_train, y_train)

In [ ]:
# 4. Generate a prediction

pred_tree = tree.predict(X_test); pred_tree

<font color = "darkgreen">
##### 2. SVM (Support Vector Machine) #####
<img align="left" src="https://78.media.tumblr.com/0e459c9df3dc85c301ae41db5e058cb8/tumblr_inline_n9xq5hiRsC1rmpjcz.jpg"
width=500 height=500 alt="SVM">

<font color = "blue">
***[sklearn.svm.SVC()](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)***

In [ ]:
from sklearn.svm import SVC 
svm = SVC(random_state=0)
svm.fit(X_train, y_train)

<font color = "darkgreen">
##### 3. Neural Networks (Multi-layer Perceptron) #####
</font>  
<img align="left" src="https://3.bp.blogspot.com/-vYvrkXroyDY/WdObH7y6rBI/AAAAAAAAAZ0/nqYZCwLD2doT354ZJLI0HBkcY-Pjc8LqACLcBGAs/s1600/neural_nets_thumbnail.png" width=400 height=300 alt="NN">
<img align="left" src="https://cdn-images-1.medium.com/max/479/1*QVIyc5HnGDWTNX3m-nIm9w.png" width=600 height=400 alt="NN">

단지 클릭만으로 신경망 체험하기: [Neural Network Playground 실습](https://developers.google.com/machine-learning/crash-course/introduction-to-neural-networks/playground-exercises?hl=ko)

<font color = "blue">
***[sklearn.neural_network.MLPClassifier()](http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)***

In [4]:
# 아래에 Neural Network 모형을 생성하는 코드를 작성하시오.
# 단, 모형을 저장하는 변수명을 mlp로 할 것!

from sklearn.neural_network import MLPClassifier 
mlp = MLPClassifier()
mlp.fit(X_train, y_train)

NameError: name 'X_train' is not defined

### Assess Model ###

<font color = "darkgreen">
##### 1. Decision Trees #####

In [ ]:
# 5. Evaluate the estimator
# 

display(tree.score(X_train, y_train))
display(tree.score(X_test, y_test))

<font color = "darkgreen">
##### 2. SVM

In [ ]:
display(svm.score(X_train, y_train))
display(svm.score(X_test, y_test))

<font color = "darkgreen">
##### 3. Neural Networks

In [ ]:
# 아래에 Neural Network 모형의 성능을 평가하는 코드를 작성하시오.

display(mlp.score(X_train, y_train))
display(mlp.score(X_test, y_test))

## Step 5: Evaluation ##

<font color = "red">
- *Which model is the best ?*
- *Is the model useful ?*
<font>

In [ ]:
best_model = tree   # Change this code if the best model is not decision tree.
best_model.score(X_test, y_test)

In [ ]:
# 랜덤 모형의 성능과 개발 모형의 성능을 비교한다.

from sklearn.dummy import DummyClassifier
print(y_test.value_counts())
DummyClassifier(strategy='most_frequent').fit(X_train, y_train).score(X_test, y_test)

## Step 6: Deployment ##

In [ ]:
# You must do the same preprocessing as the modeling data.

ndf = new.copy()
ndf['realincome'] = np.where(ndf['children']==0, ndf['income'], ndf['income']/ndf['children'])
ndf = ndf.drop(columns, axis=1)
ndf.head()

### A Case: Apply the best model to select target customers ###

In [ ]:
# 개발 모형에 고객 데이터를 적용하여 개인연금 구매여부를 예측: id 제외

ndf['pred'] = best_model.predict(ndf.loc[:,'age':'realincome'])

In [ ]:
# 개인연금 구매확률을 예측: predict_proba() 사용
print(best_model.predict_proba(ndf.loc[:,'age':'realincome']))
ndf['pred_prob'] = best_model.predict_proba(ndf.loc[:,'age':'realincome'])[:,1]
ndf.head()

In [ ]:
# 특정 조건을 만족하는 고객 리스트를 추출하고 저장
# query 조건 만족 행을 뽑을 때 사용

target = ndf.query('pred == 1 & pred_prob > 0.7')  # PEP에 가입할 확율이 70%가 넘는 고객만 추출
target.sort_values(by="pred_prob", ascending=False).to_csv("pep_target.csv", index=False)
pd.read_csv("pep_target.csv").tail()